# Word2Vec for Clinical Codes Embedding & Logistic Regression 1-year mortality prediction

**Goal**: Move from sparse raw counts to **dense, meaningful embeddings** of medical codes.

Train our own medical Word2Vec model

Evaluate embeddings in 3 ways (intrinsic, visualization, extrinsic)

# 📂 **Data Access & Setup**

This project uses the **MIMIC-IV v3.1** dataset. Due to the sensitive nature of clinical data and regulation, the raw datasets are not included in this repository.

**1. Requesting Access**
To run this pipeline, you must have a signed Data Use Agreement (DUA):

Training: Complete the [CITI Data or Specimens Researchers training](https://about.citiprogram.org/)

PhysioNet: Create an account and request access via the [MIMIC-IV PhysioNet Page](https://physionet.org/content/mimiciv/3.1/)

**2. Local Setup**
Once access is granted, download the following files and place them in a folder named data/ in the root of this project:

hosp/patients.csv.gz

hosp/admissions.csv.gz

hosp/diagnoses_icd.csv.gz

hosp/procedures_icd.csv.gz

icu/icustays.csv.gz

**3. Running on Google Colab**
If using Google Colab, upload these files to your Google Drive and update the data_path variable at the top of the notebook to point to your Drive folder.

## 0. Install & Imports

In [ ]:
!pip install -q gensim scikit-learn pandas numpy matplotlib seaborn tqdm

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm # A nice progress bar
tqdm.pandas()

# Traditional ML
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score, classification_report, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Embedding specific
from gensim.models import Word2Vec

# Visualization
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## 1. The Problem with Traditional Representations

In [ ]:
codes = [
    ['250001'], # Type 1 diabetes
    ['250000'], # Type 2 diabetes
    ['R51']  # Headache
]
# sklearn's OneHotEncoder
ohe = OneHotEncoder(sparse_output=False).fit(codes)
vec = ohe.transform(codes)

# check the cosine similarity
print("Cosine similarity with one-hot:")
print(f"  Type1 DM vs Type2 DM = {cosine_similarity(vec[0:1], vec[1:2])[0][0]:.3f}")
print(f"  Type1 DM vs Headache = {cosine_similarity(vec[0:1], vec[2:3])[0][0]:.3f}")

## 2. Build MIMIC-IV Corpus

We will load all codes, group them by `hadm_id`, and create a list-of-lists. This will be our corpus.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


DATA_PATH = '' # write your data path

if os.path.exists(DATA_PATH):
    print("✅ Path found! Loading files...")
else:
    print("❌ Path still not found. Try running the debug script below.")

diag = pd.read_csv(os.path.join(DATA_PATH, "diagnoses_icd.csv.gz"))
proc = pd.read_csv(os.path.join(DATA_PATH, "procedures_icd.csv.gz"))

diag = diag[['hadm_id', 'icd_code']].rename(columns={'icd_code': 'code'})
proc = proc[['hadm_id', 'icd_code']].rename(columns={'icd_code': 'code'})

all_codes = pd.concat([diag, proc]).dropna()
corpus_df = all_codes.groupby('hadm_id')['code'].apply(list)

# Convert the pandas Series to a simple list of lists
corpus = corpus_df.tolist()


In [ ]:
num_sentences = len(corpus)
num_words = sum(len(x) for x in corpus)

vocabulary = set(code for admission in corpus for code in admission)
vocab_size = len(vocabulary)

print(f"  Number of Sentences (Admissions): {num_sentences:,}")
print(f"  Total Number of Words (Codes):    {num_words:,}")
print(f"  Vocabulary Size (Unique Codes):   {vocab_size:,}")

## 3. Train Med2Vec

Now that we have our corpus, we can train a Word2Vec model using gensim.

In [ ]:
model_file = "med2vec_mimic4.w2v"

if not os.path.exists(model_file):
    print("Training model... (~10 min)")
    model = Word2Vec(
        sentences=corpus,
        vector_size=100,
        window=5,
        sg=1,
        negative=10,
        min_count=5,
        epochs=10,
        workers=os.cpu_count()-1,
        seed=42
    )
    model.save(model_file)
else:
    model = Word2Vec.load(model_file)

#The trained word vectors are stored in a KeyedVectors instance, as model.wv
wv = model.wv

## 4. Intrinsic Evaluation

In [ ]:
# Load official code → name mappings
d_diag = pd.read_csv(os.path.join(DATA_PATH, "d_icd_diagnoses.csv.gz"))
d_proc = pd.read_csv(os.path.join(DATA_PATH, "d_icd_procedures.csv.gz"))

# Create dictionaries
diag_map = dict(zip(d_diag['icd_code'], d_diag['long_title']))
proc_map = dict(zip(d_proc['icd_code'], d_proc['long_title']))


def code_to_name(code):
    if code in diag_map:
        return diag_map[code][:70]  # truncate for display
    if code in proc_map:
        return proc_map[code][:70]
    return code  # fallback

# Enhanced similar() function with names
def similar(code, n=8):
    if code not in wv:
        print(f"'{code}' not in vocabulary")
        return
    print(f"\nMost similar to {code}  |  {code_to_name(code)}")
    print("─" * 90)
    for w, s in wv.most_similar(code, topn=n):
        name = code_to_name(w)
        print(f"  {w:10}  {s:.4f}  |  {name}")

# Show results
similar('25000')   # Type 2 diabetes mellitus
similar('25001')   # Type 1 diabetes mellitus
similar('I10')     # Essential hypertension

## 5. PCA Visualization

In [ ]:
vocab = list(wv.key_to_index.keys())
X = wv[vocab]
pca = PCA(n_components=2, random_state=42)
X_2d = pca.fit_transform(X)


vis_df = pd.DataFrame({
    'code': vocab,
    'x': X_2d[:, 0],
    'y': X_2d[:, 1],
    'name': [code_to_name(c) for c in vocab],
    'icd': ['ICD-10' if c and c[0].isalpha() else 'ICD-9' for c in vocab]

})

#Define clinical groups
diabetes_codes     = [c for c in vocab if c.startswith('250') or c.startswith(('E08','E09','E10','E11','E13'))]
hypertension_codes = [c for c in vocab if c.startswith('401') or c.startswith(('I10','I11','I12','I13','I15'))]
breast_cancer_codes = [c for c in vocab if c.startswith(('174','175')) or c.startswith('C50')]
heart_failure_codes = [c for c in vocab if c.startswith('428') or c.startswith('I50')]


# Assign groups
vis_df['group'] = 'Other'
vis_df.loc[vis_df['code'].isin(diabetes_codes), 'group'] = 'Diabetes'
vis_df.loc[vis_df['code'].isin(hypertension_codes), 'group'] = 'Hypertension'
vis_df.loc[vis_df['code'].isin(breast_cancer_codes), 'group'] = 'Breast Cancer'
vis_df.loc[vis_df['code'].isin(heart_failure_codes), 'group'] = 'Heart Failure'

# Color palette
palette = {
    'Other': 'lightgray',
    'Diabetes': '#d62728',          # red
    'Hypertension': '#1f77b4',      # blue
    'Breast Cancer': '#ff7f0e',     # orange
    'Heart Failure': '#2ca02c',     # green
    'Key Medications': '#9467bd'    # purple
}

## Define type of codes
proc_set = d_proc['icd_code'].unique().tolist()
diag_set = d_diag['icd_code'].unique().tolist()

def def_code_type(x):
    if x in diag_set:
        return "Diagnosis"
    elif x in proc_set:
        return "Procedures"
    else:
        return "Other"
vis_df['type'] = vis_df["code"].apply(def_code_type)



# Plot codes of interest
plt.figure(figsize=(8, 8))
sns.scatterplot(
    data=vis_df[vis_df['group'] == 'Other'],
    x='x', y='y',
    color='lightgray', alpha=0.4, s=50, label='Other codes'
)

sns.scatterplot(
    data=vis_df[vis_df['group'] != 'Other'],
    x='x', y='y',
    hue='group',
    palette=palette,
    s=80,
    edgecolor='black',
    linewidth=0.8
)

plt.title('Med2Vec Embeddings')
plt.xlabel(f'PC1')
plt.ylabel(f'PC2')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', title='Clinical Group')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot colored by code type
plt.figure(figsize=(8, 8))
sns.scatterplot(data=vis_df, x='x', y='y', hue='type', s=50, alpha=0.8)

plt.title('Med2Vec Embeddings')
plt.xlabel(f'PC1')
plt.ylabel(f'PC2')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', title='Clinical Group')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot colored by code vocabulary
plt.figure(figsize=(8, 8))
sns.scatterplot(data=vis_df, x='x', y='y', hue='icd', s=50, alpha=0.8)

plt.title('Med2Vec Embeddings')
plt.xlabel(f'PC1')
plt.ylabel(f'PC2')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', title='Clinical Group')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
pca_95 = PCA(n_components=0.95)
X_reduced = pca_95.fit_transform(X)

n_components_95 = pca_95.n_components_
total_dims = X.shape[1]

print(f"Original Dimensions: {total_dims}")
print(f"Components needed for 95% variance: {n_components_95}")
print(f"Percentage of dimensions utilized: {n_components_95/total_dims:.1%}")

# Cumulative Variance
pca_full = PCA().fit(X)
plt.figure(figsize=(8, 5))
plt.plot(np.cumsum(pca_full.explained_variance_ratio_))
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Explained Variance')
plt.axvline(x=n_components_95, color='r', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Cumulative Variance')
plt.legend()
plt.grid(True)
plt.show()

## 6. Extrinsic Evaluation: LR 1-Year Mortality Prediction

In [ ]:
# Rebuild exact same cohort as previous notebook (L1-LR 1-year mortality prediction)
patients = pd.read_csv(os.path.join(DATA_PATH, "patients.csv.gz"))
admissions = pd.read_csv(os.path.join(DATA_PATH, "admissions.csv.gz"))
icustays = pd.read_csv(os.path.join(DATA_PATH, "icustays.csv.gz"))

for df in [patients, admissions, icustays]:
    for col in df.columns:
        if 'time' in col.lower() or col == 'dod':
            df[col] = pd.to_datetime(df[col])

adm = admissions.merge(patients[['subject_id','anchor_age','anchor_year','dod']], on='subject_id')
adm['age'] = adm['anchor_age'] + (adm['admittime'].dt.year - adm['anchor_year'])

#Keep adults and exclude patients dead in first hospital admin
icu = icustays.merge(adm[adm['age'] >= 18], on=['subject_id','hadm_id'])
first_stay = icu.sort_values('intime').groupby('subject_id').first().reset_index()
cohort = first_stay[first_stay['deathtime'].isna()].copy()
cohort['days_to_death'] = (cohort['dod'] - cohort['outtime']).dt.days
cohort['label'] = ((cohort['days_to_death'] <= 365) & (cohort['days_to_death'] > 0)).astype(int)

data = cohort[['subject_id', 'hadm_id', 'label']]
print(f"Final cohort: {len(data)} patients")

In [ ]:
corpus = corpus_df.to_dict()
def patient_vector(hadm):
    codes = corpus.get(hadm, [])          # corpus must be dict
    vecs = [wv[c] for c in codes if c in wv]
    if vecs:
        return np.mean(vecs, axis=0)
    else:
        return np.zeros(wv.vector_size)

# Build X: ensure order matches data['hadm_id']
X = np.stack(data['hadm_id'].progress_apply(patient_vector).values)
y = data['label'].values

# Diagnostic: fraction of all-zero vectors
n_zero = np.sum(np.all(X == 0, axis=1))
print(f"Patient vectors: {X.shape}, zero vectors: {n_zero} ({n_zero/len(X):.2%})")


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


model = LogisticRegression(max_iter=1000, solver='lbfgs')
model.fit(X_train, y_train)


y_probs = model.predict_proba(X_test)[:, 1]

auroc = roc_auc_score(y_test, y_probs)
auprc = average_precision_score(y_test, y_probs)

print(f"Results for Patient Embeddings (Mortality Prediction):")
print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f"AUROC (Area Under ROC Curve):        {auroc:.4f}")
print(f"AUPRC (Area Under Precision-Recall): {auprc:.4f}")
print(f"Baseline AUPRC (Random Guess):       {y.mean():.4f}")
print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# ROC Curve
from sklearn.metrics import roc_curve, precision_recall_curve
fpr, tpr, _ = roc_curve(y_test, y_probs)
ax1.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {auroc:.2f})')
ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('Receiver Operating Characteristic (ROC)')
ax1.legend(loc="lower right")
ax1.grid(alpha=0.3)

# PR Curve
precision, recall, _ = precision_recall_curve(y_test, y_probs)
ax2.plot(recall, precision, color='green', lw=2, label=f'PR (AUC = {auprc:.2f})')
ax2.axhline(y=y.mean(), color='navy', linestyle='--', label=f'Baseline ({y.mean():.2f})')
ax2.set_xlabel('Recall (Sensitivity)')
ax2.set_ylabel('Precision (PPV)')
ax2.set_title('Precision-Recall Curve')
ax2.legend(loc="upper right")
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()